In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column as fc 
from tensorflow.keras.datasets import boston_housing

%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [3]:
tf.__version__

'2.6.0'

In [4]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [5]:
features = ['CRIM', 'ZN', 'INDUS','CHAS','NOX','RM','AGE',
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
 
x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [6]:
feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)


[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTRATIO', shape=(1

In [7]:
def estimator_input_fn(df_data, df_label, epochs=20, shuffle=True, batch_size=64):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

train_input_fn = estimator_input_fn(x_train_df, y_train_df)
val_input_fn = estimator_input_fn(x_test_df, y_test_df, epochs=1, shuffle=False)

In [12]:
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir = 'logs/func/')
linear_est.train(train_input_fn, steps=100)
result = linear_est.evaluate(val_input_fn)
print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs/func/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflo

/Users/jonasgacrama/miniforge3/envs/tfcpu/lib/python3.9/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into logs/func/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:loss = 62.524963, step = 100
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 200...
INFO:tensorflow:Saving checkpoints for 200 into logs/func/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 200...
INFO:tensorflow:Loss for final step: 52.652687.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-09T20:26:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/func/model.ckpt-200


/Users/jonasgacrama/miniforge3/envs/tfcpu/lib/python3.9/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.15505s
INFO:tensorflow:Finished evaluation at 2021-11-09-20:26:15
INFO:tensorflow:Saving dict for global step 200: average_loss = 64.2033, global_step = 200, label/mean = 23.078432, loss = 64.68512, prediction/mean = 21.500002
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 200: logs/func/model.ckpt-200
{'average_loss': 64.2033, 'label/mean': 23.078432, 'loss': 64.68512, 'prediction/mean': 21.500002, 'global_step': 200}


In [9]:
result = linear_est.predict(val_input_fn)

In [10]:
for pred,exp in zip(result, y_test[:32]):
    print("Predicted Value: ", pred['predictions'][0], "Expected: ", exp)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/func/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


/Users/jonasgacrama/miniforge3/envs/tfcpu/lib/python3.9/site-packages/keras/engine/base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Predicted Value:  4.2978034 Expected:  7.2
Predicted Value:  22.783546 Expected:  18.8
Predicted Value:  21.17855 Expected:  19.0
Predicted Value:  23.103128 Expected:  27.0
Predicted Value:  21.798296 Expected:  22.2
Predicted Value:  21.105312 Expected:  24.5
Predicted Value:  28.716532 Expected:  31.2
Predicted Value:  24.755354 Expected:  22.9
Predicted Value:  20.073542 Expected:  20.5
Predicted Value:  23.220179 Expected:  23.2
Predicted Value:  9.118159 Expected:  18.6
Predicted Value:  22.261189 Expected:  14.5
Predicted Value:  22.612947 Expected:  17.8
Predicted Value:  22.119242 Expected:  50.0
Predicted Value:  18.271633 Expected:  20.8
Predicted Value:  23.99715 Expected:  24.3
Predicted Value:  19.925327 Expected:  24.2
Predicted Value:  20.698977 Expected:  19.8
Predicted Value:  23.062147 Expected:  19.1
Predicted Value:  25.614574 Expected:  22.7
Predicted Value:  19.152197 Expected:  12.0
Predicted Value:  5.186852 Expected:  10.2
Predicted Value:  20.844763 Expected:

In [11]:
%tensorboard --logdir logs/func